In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
import glob
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from io import BytesIO
from PIL import Image
# from kaggle_datasets import KaggleDatasets
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!pip install kaggle_datasets

ERROR: Could not find a version that satisfies the requirement kaggle_datasets (from versions: none)
ERROR: No matching distribution found for kaggle_datasets


In [ ]:
IMAGE_SIZE = [220, 220]
EPOCHS = 50
SEED = 1447
CLASSES = 10
BATCH_SIZE = 50

In [ ]:
train_dir = '/content/gdrive/MyDrive/Monkey/training/training'
val_dir = '/content/gdrive/MyDrive/Monkey/validation/validation'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    class_mode='categorical'
)

Found 1098 images belonging to 10 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(
    val_dir, 
    target_size=IMAGE_SIZE, 
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=False,
    class_mode='categorical'
)

Found 272 images belonging to 10 classes.


In [ ]:
resnet_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False
)

x = resnet_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512,activation='relu')(x)
predictions = tf.keras.layers.Dense(
   10,
    activation='softmax'
)(x)

resnet50_model = tf.keras.models.Model(
    inputs= resnet_model.input, 
    outputs=predictions
)

resnet50_model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy']
)

resnet_history = resnet50_model.fit_generator(
    train_generator, 
    epochs=EPOCHS, 
    shuffle=True, 
    verbose=1,
    validation_data=validation_generator
)

94773248/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
22/22 [==============================] - 329s 14s/step - loss: 2.4231 - accuracy: 0.1576 - val_loss: 2.4903 - val_accuracy: 0.1029
Epoch 2/50
22/22 [==============================] - 47s 2s/step - loss: 2.1595 - accuracy: 0.2450 - val_loss: 2.4177 - val_accuracy: 0.1029
Epoch 3/50
22/22 [==============================] - 47s 2s/step - loss: 1.9837 - accuracy: 0.3097 - val_loss: 2.3859 - val_accuracy: 0.1029
Epoch 4/50
22/22 [==============================] - 47s 2s/step - loss: 1.7771 - accuracy: 0.4718 - val_loss: 2.4185 - val_accuracy: 0.0993
Epoch 5/50
22/22 [==============================] - 47s 2s/step - loss: 1.6131 - accuracy: 0.5619 - val_loss: 2.5563 - val_accuracy: 0.1176
Epoch 6/50
22/22 [==============================] - 47s 2s/step - loss: 1.4522 - accuracy: 0.6448 - val_loss: 2.8449 - val_accuracy: 0.1029
Epoch 7/50
22/22 [==============================] - 47s 2s/step - loss: 1.3135 - accuracy: 0.7040 - val_loss: 3.1439 - val_accuracy: 0.1103
Epoch 8/50
22/22 [

In [ ]:
resnet50_model.save_weights('resnet50_weight_monkey.h5')